### 自定义NB模型
#### 输入
- X_train,Y_train,X_test,Y_test
    - 其中，X为文本特征向量化处理后(CountVectorizer)的array数组，Y为X对应的分类标签list，X的行业与Y一一对应
    
#### 输出
- X_test各样本的预测分类结果Y_predict，以及分类准确率

#### 过程
- 利用训练集各特征词出现的频率和对应标签概率，训练NB模型各概率参数
- 求测试集各特征在训练集对应的先验概率
- 将测试集各特征在训练集对应的先验概率乘以条件概率P(Y=ck)，得到测试集各样本后验概率，取后验概率最大的标签类别为该测试样本类别

#### 1. 利用训练集，训练概率参数（拉普拉斯平滑）[类似mnb.fit()]
- 条件概率：P(Y=ck)
- 先验概率：P(X1=0|Y=ck),P(X1=1|Y=ck),P(X2=0|Y=ck)……

#### 2. 将测试集各特征向量值带入训练的概率参数中，计算后验概率，取使后验概率最大的Y=ck为测试样本的分类[类似mnb.predict(), mnb.predict_proba()]
- 测试集样本特征向量为0时，不将刚才训练的对应概率参数纳入计算
- 测试集样本特征向量>=1时(即测试样本出现该特征向量的词)，将刚才训练的特征向量对应的概率参数纳入计算
- 分别计算垃圾邮件下和正常邮件下每个样本的后验概率，取后验概率最大的类别为样本分类

#### 3. 计算分类准确率 [类似mnb.score()]

In [30]:
"""
输入：X_train,Y_train,X_test,Y_test
    其中，X为文本特征向量化处理后(CountVectorizer)的array，Y为X对应的分类标签list，XY一一对应
输出：X_test各样本的预测分类结果Y_predict，分类准确率
    其中，0-正常邮件，1-垃圾邮件
"""
def wheel_nb(X_train,Y_train,X_test,Y_test):
    import pandas as pd
    import numpy as np
    import re
    
    #先将训练集的内容和标签合为一个dataframe
    d={"content":X_train.tolist(),"label":Y_train}
    emails_train=pd.DataFrame(data=d)

    #将正常邮件(Y=0)和垃圾邮件(Y=1)分为两个子集
    normal=emails_train[emails_train.label==0]
    normal.reset_index(inplace=True,drop=True) #重置normal索引，作用于原表，丢弃之前的索引
    spam=emails_train[emails_train.label==1]
    spam.reset_index(inplace=True,drop=True) #重置spam索引，作用于原表，丢弃之前的索引

    """计算Y_train=0、1的条件概率（拉普拉斯平滑）"""
    Py0=(len(normal)+1)/(len(emails_train)+2)
    Py1=(len(spam)+1)/(len(emails_train)+2)

    """计算X_train各特征向量取各特征值时的先验概率（拉普拉斯平滑）"""
    """计算垃圾邮件中,各特征向量的先验概率"""
    vd=len(spam.content[0]) #特征向量的维度
    spam_count_dict={} #用于保存content特征向量按列累加的结果
    spam_count_prob={} #用于保存垃圾邮件中各特征向量出现的概率

    #求content各特征向量按列累加的结果，用于计算各向量在训练集中出现的概率
    for i in range(len(spam)):
        for j in range(vd):
            spam_count_dict[j]=spam_count_dict.get(j,0)+spam.content[i][j] #计算垃圾邮件中各特征向量出现的次数，即，求content各特征向量count按列累加的结果

    for j in range(vd):
        spam_count_prob[j]=(spam_count_dict.get(j,0)+1)/(len(spam)+2)#计算垃圾邮件中各特征向量出现的概率（拉普拉斯平滑）

    """计算正常邮件中,各特征向量的先验概率"""
    normal_count_dict={} #用于保存content特征向量按列累加的结果
    normal_count_prob={} #用于保存正常邮件中各特征向量出现的概率

    #求content各特征向量按列累加的结果，用于计算各向量在训练集中出现的概率
    for i in range(len(normal)):
        for j in range(vd):
            normal_count_dict[j]=normal_count_dict.get(j,0)+normal.content[i][j] #计算垃圾邮件中各特征向量出现的次数，即，求content各特征向量count按列累加的结果

    for j in range(vd):
        normal_count_prob[j]=(normal_count_dict.get(j,0)+1)/(len(normal)+2)#计算垃圾邮件中各特征向量出现的概率（拉普拉斯平滑）

    """计算各测试样本的后验概率"""
    test_classify={} #用于保存测试集各样本的后验概率 P(Y|X)=P(Y)*P(X|Y)/P(X)
    Px_spam={} #用于保存测试集各样本在垃圾邮件下的先验概率 P(X|Y)
    Px_normal={} #用于保存测试集各样本在正常邮件下的先验概率 P(X|Y)

    for i in range(X_test.shape[0]):
        for j in range(X_test.shape[1]):
            if X_test[i][j]!=0:
                Px_spam[i]=Px_spam.get(i,1)*spam_count_prob.get(j)#计算垃圾邮件下，各测试样本的后验概率
                Px_normal[i]=Px_normal.get(i,1)*normal_count_prob.get(j)#计算正常邮件下，各测试样本的后验概率

        test_classify[i]=Py0*Px_normal.get(i,0),Py1*Px_spam.get(i,0) #后验概率P(Y|X)=P(Y)*P(X|Y)/P(X)

    #比较各样本属于不同分类时(正常/垃圾)的后验概率，去后验概率大的为样本分类结果
    results={} #用于存放邮件判定结果
    for key,value in test_classify.items():
        if value[0]<=value[1]: #value[0]-样本为正常邮件的后验概率，value[1]-样本为垃圾邮件的后验概率
            results[key]=1
        else:
            results[key]=0

    """计算分类准确率"""
    count=0 #计数，统计被正确分类的邮件数量
    for key,value in results.items():
        if value==Y_test[key]:
            count+=1
    score=count/len(Y_test)
    
    print ("测试样本预测分类为(按索引排序)：")
    print (results.values(),"\n")
    print ("测试样本实际分类为(按索引排序)：")
    print (Y_test,"\n")
    print ("NB模型分类准确率为：{0}%".format(score*100))

    return results,score

### 测试NB模型

In [36]:
#读取数据
emails=pd.read_csv(r"E:\python\data\emails_spam.csv",encoding="utf-8")
emails.head(9)

,content,type
0,"招商银行信用卡电子账单2018年6月?-?07/13?￥1,540.00?＄?0.00?￥1...",0
1,密码重置邮件-来自智联招聘?,0
2,信用管家消费提醒?-?尊敬的邓莎女士：?您好，感谢您选择招商银行信用卡！?￥2189?￥58...,0
3,Apple 提供的收据?-?收据?APPLE?ID?348708632@qq.com付款信息...,0
4,信用管家消费提醒?-?尊敬的邓莎女士：?您好，感谢您选择招商银行信用卡！?￥1540?￥64...,0
5,6月20日徐晨阳《硅谷创新机制解密》报告?-?各位校友：?通知请见：https://www....,0
6,中国科学技术大学六十周年校庆纪念活动 校友邀请函?-??尊敬的校友：?您好！红专并进一甲子，...,0
7,少女心晒一“夏”，ELLE Club等你解锁夏季最潮玩法！（?-?如果您不能正常浏览此邮件，...,1
8,网上购票系统--用户支付通知?-??尊敬的?邓女士：?您好！?您于2018年06月04日在中...,0


In [37]:
#清洗数据
def text_format():
    import jieba
    import re
    import pandas as pd
    
    print ("待处理文本格式要求:utf-8编码格式，仅包含待处理文本，每行为一条文本")
    text_path=input("请输入待清洗文本路径+名字：")
    
    #加载用户自定义词典用于分词
    userdict_path=input("请输入自定义分词词典路径+名字(可不输入)：")
    if userdict_path !="":
        jieba.load_userdict(userdict_path)
        
    #根据用户输入地址，读取文件
    with open(text_path,"r",encoding="utf-8") as file:
        text=file.readlines()
    for i in range(len(text)):
        text[i]=text[i].strip()
    
    #定义一个空列表，用于存放分词后的文本，长度和text一致
    text_word=[[] for i in range(len(text))]
    
    splitter=re.compile(r"\W+|\d+|[a-z]+") #正则匹配，去除文本中的符号、数字、字母等非中文字符的元素
    for i in range(len(text)):
        text[i]=splitter.split(text[i].lower())
        text[i]=[word for word in text[i] if len(word)>1] #每条文本已经被分为一段一段的句子，每条文本此时是一个list，先去除其中字段长度小于等于1的单词
        for word in text[i]:
            text_word[i].extend(jieba.lcut(word))
        text_word[i]=" ".join(text_word[i]) #为了便于TfidfVectorizer等文本向量化处理，将每条标题用元素用空格连起来
    
    return text_word

In [38]:
emails_format=text_format()

待处理文本格式要求:utf-8编码格式，仅包含待处理文本，每行为一条文本
请输入待清洗文本路径+名字：E:\python\data\emails_spam.txt
请输入自定义分词词典路径+名字(可不输入)：


In [39]:
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

#建立训练集、测试集
label=emails.type.tolist()
X_train,X_test,Y_train,Y_test=train_test_split(emails_format,label,test_size=0.2,random_state=7)

#加载并处理停用词典
with open(r"E:\python\data\stopwords.txt","r",encoding="utf-8") as file:
    stop_words=file.readlines()
for i in range(len(stop_words)):
    stop_words[i]=stop_words[i].strip("\n")
    
#构成词袋模型，记录各个词出现的次数
cv=CountVectorizer(stop_words=stop_words)
X_train_count=cv.fit_transform(X_train)
X_test_count=cv.transform(X_test)

#### 将数据带入NB模型进行测试

In [40]:
#将数据带入NB模型进行测试
wheel_nb(X_train_count.toarray(),Y_train,X_test_count.toarray(),Y_test)

测试样本预测分类为(按索引排序)：
dict_values([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]) 

测试样本实际分类为(按索引排序)：
[0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0] 

NB模型分类准确率为：86.66666666666667%


({0: 0,
  1: 0,
  2: 0,
  3: 1,
  4: 0,
  5: 1,
  6: 0,
  7: 0,
  8: 0,
  9: 0,
  10: 0,
  11: 0,
  12: 0,
  13: 0,
  14: 0},
 0.8666666666666667)